 #  **Final Project Report**

**Title** : Web-Based Scientific Calculator Using Flask.

**Submitted by** : Prem Sunil Peherkar


### **Objectives**

-This Jupyter notebook contains a complete Flask-based scientific calculator application. 

-It incudes designing and developing a scientific calculator  using Python's Flask framework.

-It offers both standard and scientific operations including trigonometric, logarithmic, exponential, and arithmetic computations.

-It features a keyboard-enabled interface secure evaluation of expressions and a responsive user-friendly layout—all running in the browser.
 

#### **Tools & Technologies**

| Tool/Technology | Purpose |
|------------------|---------|
| Python 3 | Programming language |
| Flask | Lightweight web framework |
| HTML/CSS | Frontend structure and styling |
| JavaScript | Keyboard event handling |
| Math module | Scientific calculations |




### **System Architecture**

 ### **Frontend**:
- **HTML**: Layout and form structure
- **CSS**: Styling of calculator UI
- **JavaScript**: Captures keyboard events, handles 'Enter', 'Backspace'.

### **Backend (Flask):**
- Receives inputs from user
- Securely evaluates expressions using a whitelisted environment
- Returns the result back to be displayed on the calculator screen


In [1]:
pip install flask 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import annotations
import math
from flask import Flask, render_template_string, request


##  **Key Features**

###  Scientific Capabilities:
- Trigonometric : 'sin(x)', 'cos(x)', 'tan(x)'
- Logarithmic : 'log(x)'
- Exponential : 'exp(x)', '^' (power)
- Root : 'sqrt(x)'
- Constants : 'pi', 'e'
- Operators : '+', '-', '*', '/', '%', '^', '()'  

###  Secure Expression Evaluation:
- Expressions are parsed and safely evaluated using only 'math' module functions.
- Built-in functions or other Python code execution is **blocked**.

###  Keyboard Support:
- Use numbers and operators directly from the keyboard.
- Press 'Enter' to evaluate.
- Backspace' to delete characters.


In [3]:
app = Flask(__name__)
authorized_names: dict[str, object] = {
    name: getattr(math, name) for name in dir(math) if not name.startswith("__")
}
authorized_names.update({
    "sin": math.sin,
    "cos": math.cos,
    "tan": math.tan,
    "log": math.log,
    "sqrt": math.sqrt,
    "pi": math.pi,
    "e": math.e,
})

In [4]:
def safe_eval(expr: str) -> float:
    """Evaluate *expr* using only math‑module names defined above."""
    return eval(expr, {"__builtins__": {}}, authorized_names)


 **Note:** The HTML string contains all the calculator layout, style, and JavaScript needed for keyboard support. It uses render_template_string() to serve it directly.
   

In [5]:
HTML_TEMPLATE: str = """<!DOCTYPE html>
<html lang=\"en\">
<head>
<meta charset=\"UTF-8\">
<meta name=\"viewport\" content=\"width=device-width,initial-scale=1\">
<title>Scientific Calculator</title>
<style>
  :root{--bg:#202124;--card:#2d2f31;--btn:#3c4043;--btn-alt:#8ab4f8;--green:#34a853;--red:#ea4335;--text:#fff;}
  *{box-sizing:border-box;font-family:system-ui,Segoe UI,Roboto,Helvetica,Arial,sans-serif}
  body{margin:0;display:flex;justify-content:center;align-items:center;height:100vh;background:var(--bg);color:var(--text);}
  #calc{width:auto;padding:20px;background:var(--card);border-radius:16px;box-shadow:0 8px 20px rgba(0,0,0,.55);}
  #display{width:100%;height:70px;font-size:34px;text-align:right;padding:10px 14px;margin-bottom:14px;border:0;border-radius:12px;background:var(--bg);color:var(--text);outline:none;}
  .row{display:flex;justify-content:space-between;}
  button{flex:1;margin:4px;height:40px;font-size:20px;border:0;border-radius:20em;background:var(--btn);color:var(--text);cursor:pointer;transition:transform .1s;}
  button.operator{background:var(--btn-alt);color:#000;}
  button.equal{background:var(--green);}button.clear{background:var(--red);}button:active{transform:scale(.97);}  
</style>
</head>
<body>
  <form id=\"form\" method=\"POST\" autocomplete=\"off\">
    <div id=\"calc\">
      <input id=\"display\" name=\"expression\" value=\"{{ expression }}\" autofocus>
      {% for r in buttons %}
      <div class=\"row\">
        {% for lbl in r %}
          {% set cls = (lbl in operators) and 'operator' or '' %}
          {% set cls = (lbl=='=') and 'equal' or cls %}
          {% set cls = (lbl=='C') and 'clear' or cls %}
          <button type=\"button\" class=\"{{ cls }}\" onclick=\"press('{{ lbl }}')\">{{ lbl }}</button>
        {% endfor %}
      </div>
      {% endfor %}
      <input type=\"hidden\" id=\"btn\" name=\"btn\">
    </div>
  </form>
<script>
function press(v){document.getElementById('btn').value=v;document.getElementById('form').submit();}
const allowed = /[0-9()+\-*/.%^]/;
document.addEventListener('keydown', e=>{
  if(e.key==='Enter'){e.preventDefault();press('=');}
  else if(e.key==='Backspace'){e.preventDefault();press('⌫');}
  else if(allowed.test(e.key)){e.preventDefault();press(e.key);}  
});
</script>
</body>
</html>"""

<>:1: SyntaxWarning: invalid escape sequence '\-'
<>:1: SyntaxWarning: invalid escape sequence '\-'
C:\Users\Asus\AppData\Local\Temp\ipykernel_12220\3411441451.py:1: SyntaxWarning: invalid escape sequence '\-'
  HTML_TEMPLATE: str = """<!DOCTYPE html>


In [6]:
BUTTON_ROWS: list[list[str]] = [
    ["AC", "(", ")", "⌫"],
    ["7", "8", "9", "/"],
    ["4", "5", "6", "*"],
    ["1", "2", "3", "-"],
    ["0", ".", "%", "+"],
    ["pi", "e", "^", "sqrt("],
    ["sin(", "cos(", "tan(", "log("],
    ["exp(", "="]
]

OPERATORS = {"+","-","*","/","%","^"}

In [7]:
@app.route("/", methods=["GET", "POST"])
def calculator():
    expression: str = request.form.get("expression", "") if request.method == "POST" else ""
    btn: str = request.form.get("btn", "")

    if btn == "AC":
        expression = ""
    elif btn == "⌫":
        expression = expression[:-1]
    elif btn and btn != "=":
        expression += btn
    elif btn == "=":
        try:
            calc_expr = expression.replace("^", "**")
            result = safe_eval(calc_expr)
            expression = str(result)
        except Exception:
            expression = "Error"

    return render_template_string(
        HTML_TEMPLATE,
        expression=expression,
        buttons=BUTTON_ROWS,
        operators=OPERATORS,
    )

In [8]:


import threading

def run_app():
    app.run(debug=False, use_reloader=False, port=8080)

threading.Thread(target=run_app).start()

# Run the app (this will show you a public link to open the calculator)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [17/Jul/2025 07:34:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2025 07:34:10] "GET /favicon.ico HTTP/1.1" 404 -


  ### **Conclusion**
  - Built a working scientific calculator using Flask.
    
  - Implemented basic and advanced math operations.

  - Ensured safety using restricted eval. 
    
  - Added support for keyboard input and responsive UI.


### ***References***

- [Flask Documentation](https://flask.palletsprojects.com/)
- [Python math Module](https://docs.python.org/3/library/math.html)
- JavaScript Keyboard Events - chat GPT